In [1]:
%%time

#import in all libraries and methods
%run -i 'myimports.py'
%run -i 'datagathering.py'
%run -i 'nlp_processing.py' #imports packages and methods
import pymongo
main = "https://www.happyscribe.com"                
url="/public/lex-fridman-podcast-artificial-intelligence-ai"
podcast_host = "Lex Fridman"
html_location = "Archive/happyscribe_Lex_Fridman_html.txt"
# update_transcripts(html_location)

CPU times: user 15.2 s, sys: 1.98 s, total: 17.2 s
Wall time: 18.3 s


In [2]:
%%time
#convert file of podcasts to list of docs and docbin and write to disk

# %run -i 'nlp_implementation.py'
podcast_name = "Archive/both"
# os.listdir(podcast_name)
onlyfiles = folder_to_filelist(podcast_name) #get list of all files in foldername

doc_bin = DocBin(store_user_data=True) #docbin container for serialization
docs = [] #list of docs 
print("starting")
for doc, name in nlp.pipe(onlyfiles, as_tuples=True): #piping all collection of docs to make doclist and docbin
    #each doc contians hostname, guest, title, entities mentioned, and summary
    name = re.split("[\|]",name)
    name=name[1:] if name[0][1:].isdigit() else name # store name of guest and topic, add to doc user data
    
    doc.user_data["host"] = podcast_host
    doc.user_data["guest"]= str(name[0]).replace("_"," ")
    doc.user_data["title"]= str(name[1][:-4]).replace("_"," ")
    doc.user_data["entis"] = [(ent.text, ent.label_) for ent in doc.ents]
    doc = keep_ents(doc) #extract and save all entities by filtering entity extraction
    docs.append(doc)
    doc_bin.add(doc) #add doc to list and bin
    
with open("cucumber/Lex.bin", "wb") as f: #write bytestream to first_doc.bin
    f.write(doc_bin.to_bytes())
print("done")

starting


/ilab/users/hs884/.local/lib/python3.8/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /ilab/users/hs884/.local/lib/python3.8/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


done
CPU times: user 1min 32s, sys: 34.5 s, total: 2min 7s
Wall time: 2min 44s


In [7]:
%%time
#loading in previous bytestreams, needs 2 since 2 podcast folders edit later
with open("cucumber/Lex.bin", "rb") as w: 
    new_docs1 = DocBin(store_user_data=True).from_bytes(w.read())
docs1 = list(new_docs1.get_docs(nlp.vocab))
# with open("cucumber/first_doc.bin", "rb") as w: #loading in
#     new_docs1 = DocBin(store_user_data=True).from_bytes(w.read())
# docs1 += list(new_docs1.get_docs(nlp.vocab))
for doc in docs:
    print(doc.user_data["books"])
# print(docs1[5].user_data["books"])

['Star Trek', 'Mind and Cosmos', 'Oculus Rift']
['Thinking Fast and Slow', 'Man’s Search for Meaning']
[]
['Space Odyssey']
['The Art of Computer Programming', 'Bible']
[]
[]
['Love in Math']
['Ph.D.', 'Love and Math', 'The Bridge Over the Bridge']
[]
[]
['Ph.D.', 'Guitar Zero', 'Deep Learning']
['How Life Imitates Chess']
[]
[]
[]
['Ph.D.']
[]
[]
CPU times: user 1.41 s, sys: 63.4 ms, total: 1.47 s
Wall time: 1.47 s


In [ ]:
#for each category of ents, generate csv files for ents corresponding to people

att_to_csv(docs1, "people") 
# att_to_csv(docs1, "places")
# att_to_csv(docs1, "books")
# print(sorted(set([item for sublist in docs1 for item in sublist.user_data["people"]])))

In [ ]:
#generate megagraph for book ents
edges = pd.read_csv("booksEdges.csv", sep=',').drop("id", axis=1).values.tolist()
nodes = pd.read_csv('booksNodes.csv',sep=',').set_index("id").to_dict("index")

source = [nodes.get(e[0])["name"] for e in edges]
target = [nodes.get(e[1])["name"] for e in edges]
kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':np.ones(len(source))})
G=nx.from_pandas_edgelist(kg_df, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())
plt.figure(figsize=(12,12))

pos = nx.spring_layout(G)
nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

#following are other example graphviews

In [ ]:
G=nx.from_pandas_edgelist(kg_df[kg_df['target']=="Bible"], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes



nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

In [ ]:
G=nx.from_pandas_edgelist(kg_df[kg_df['source']=="Matthew Johnson"], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes



nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

In [ ]:
G=nx.from_pandas_edgelist(kg_df[(kg_df['source']=="Matthew Johnson" )| (kg_df['source']=="Ryan Hall")], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes



nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

In [ ]:
G=nx.from_pandas_edgelist(kg_df[(kg_df['target']=="Crime and Punishment" )| (kg_df['target']=="Atlas Shrugged")], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes



nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

In [ ]:
wikipedia.summary("Yaron Brook", sentences=3)
wikipedia.summary("George Hotz", sentences=3)
wikipedia.summary("Atlas Shrugged", sentences=5)